# Practica 2

## Ejercicio 1

sofmdosfm

## Ejercicio 2


In [1]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje2/"
output_dir = output_path + "eje2/"


def fmap(key, value, context):
    context.write(value, 1)
    
def fcomb(key, values, context):
    c = 0
    for v in values: 
        c += v
    context.write(key, c)
    
def fred(key,values,context):
    n = 0
    for v in values:
        n += v
    context.write(key,n)

job = Job(input_dir,output_dir,fmap,fred)
job.setCombiner(fcomb)
success = job.waitForCompletion()
print(success)

True


## Ejercicio 3

Mi lógica 

## Ejercicio 4

Utilice el dataset Libros para implementar una aplicación MapReduce que devuelva
como salida todos los párrafos que tienen una longitud mayor al promedio. 

Considerando que la entrada es del tipo <key, párrafo>. Queremos saber cuantos cual es el párrafo con la mayor longitud.
Lo que yo considero aca es que podemos implementar una solución de dos JOBS, en primer lugar el primer mapper simplemente hace algo estilo <1, [párrafo, len(párrafo)]> El reducer va a determinar el promedio, y su salida va a ser algo estilo (promedio, [párrafo,...]). Luego los mappers del siguiente Job simplemente evalúan si el párrafo es mayor que el promedio, y se lo mandan a un dummy reducer. 

In [1]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje3/"
output_dir = output_path + "eje4a/"

def fmap1 (key, values, context):
    n = len(values)
    context.write(1,[values, n])
    
def fred1 (key,values,context):
    total_len = 0
    n = 0
    p_values = []
    for value in values:
        total_len += value[1]
        n += 1
        p_values.append(value[0])
    for p in p_values:
        context.write(total_len / n, p)

job1 = Job(input_dir,output_dir,fmap1,fred1)
success = job1.waitForCompletion()
print(success)

def fmap2 (key, value, context):
    if (len(value) > float(key)):
        context.write(value, 1)
        
def fred2 (key, values, context):
    for v in values:
        context.write(v, key)

job2 = Job(output_dir,output_path + "eje4b/",fmap2,fred2)
success = job2.waitForCompletion()
print(success)  


True
True


## Ejercicio 5 

El dataset website tiene información sobre el tiempo de permanencia de sus usuarios
en cada una de las páginas del sitio. El formato de los datos del dataset es:
<id_user, id_page, time>
Implemente una aplicación MapReduce, utilizando combiners en los casos que
considere necesario, que calcule
a. La página más visitada (la página en la que más tiempo permaneció) para cada
usuario
b. El usuario que más páginas distintas visitó
c. La página más visitada (en cuanto a cantidad de visitas, sin importar el tiempo
de permanencia) por todos los usuarios.
Indique como queda el DAG del proceso completo (las tres consultas).

Supongo que planteamos tres soluciones separadas 

### A

Queremos saber cual es la pagina que mas visito cada usuario, para la primera entrada los mappers no pueden hacer mucho ya que no pueden manejar variables globales, ni tampoco saben si van a trabajar con toda la información de un usuario, lo que si podemos hacer es que el mapper a la salida informe <(id_us, id_page), time> de esta forma cada nodo reducer puede contar cuanto tiempo paso una usuario en una pagina, luego devuelve algo de la forma <(id_us, id_page), time_acum>, luego el mapper del siguiente Job lee la información la separa y la envía al reducer, uno por usuario, el cual simplemente busca el máximo. 

In [9]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje5"
output_dir1 = output_path + "eje5a/Job1/"
output_dir2 = output_path + "eje5a/Job2/"

def fmap1 (key, value, context):
    user_id = key
    data = value.split('\t')
    page_id = data[0]
    page_time = data[1]
    context.write((user_id,page_id), page_time)
    
def fred1 (key, value, context):
    user_id, page_id = key
    total_time = 0
    for v in value:
        total_time += float(v)
    
    context.write((user_id, page_id), total_time)
    
def fmap2 (key, value, context):
    user_id = key
    data = value.split('\t')
    page_id = data[0]
    total_time = data[1]
    context.write(user_id, (page_id, total_time))
    
def fred2 (key, value, context):
    max_time = 0
    max_id = 0
    for v in value: 
        act_id = v[0]
        act_time = float(v[1])
        if (act_time > max_time):
            max_time = act_time
            max_id = act_id
    context.write(key, (max_id, max_time))
    
    
job1 = Job(input_dir, output_dir1, fmap1, fred1)
print(job1.waitForCompletion())
    
job2 = Job(output_dir1, output_dir2, fmap2, fred2)
success = job2.waitForCompletion()
print(success)

True
True


### B

Queremos saber el usuario que mas paginas visito, yo voy a considerar que no importa la cantidad de paginas totales y no la cantidad de paginas diferentes. Para esto planteo una solución de dos Jobs. 

1. El mapper se encarga de simplemente dar una salida estilo <user_id, 1> esto llega a un combiner que va a sumar las apariciones en estilo <user_id, count>, el reducer termina de totalizar dicho valor, <user_id, total>
2. El mapper simplemente devuelve <1, (user_id, total)>, el reducer determina el máximo. 

In [14]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje5"
output_dir1 = output_path + "eje5b/Job1/"
output_dir2 = output_path + "eje5b/Job2/"

def fmap1 (key, value, context):
    user_id = key
    context.write(user_id, 1)

def fcom1 (key, value, context):
    count = 0
    for v in value:
        count += v
    context.write(key, count)

def fred1(key,value,context):
    total = 0
    for v in value:
        total += v
    context.write(key, total)

def fmap2 (key, value, context):
    user_id = key
    total = value
    context.write(1, (user_id, total))

def fred2 (key, value, context):
    max_visits = 0
    max_id = 0
    
    for v in value:
        act_id = v[0]
        act_visits = int(v[1])
        if (act_visits > max_visits):
            max_visits = act_visits
            max_id = act_id
    context.write(max_id, max_visits)

job1 = Job(input_dir, output_dir1, fmap1, fred1)
job1.setCombiner(fcom1)
job1.waitForCompletion()

job2 = Job(output_dir1, output_dir2, fmap2, fred2)
job2.waitForCompletion()


True

### C

Queremos saber ahora la pagina mas visitada, podemos hacer algo similar a la anterior dos jobs.

1. El mapper devuelve <page_id, 1> luego un combiner suma estos valores y el reducer determina el total.
2. El mapper ahora determina <1, (page_id, total)>, y el reducer calcula el máximo. 

In [15]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje5"
output_dir1 = output_path + "eje5c/Job1/"
output_dir2 = output_path + "eje5c/Job2/"

def fmap1 (key, value, context):
    page_id = value.split('\t')[0]
    context.write(page_id, 1)

def fcom1 (key, value, context):
    count = 0
    for v in value:
        count += v
    context.write(key, count)

def fred1(key,value,context):
    total = 0
    for v in value:
        total += v
    context.write(key, total)

def fmap2 (key, value, context):
    page_id = key
    total = value
    context.write(1, (page_id, total))

def fred2 (key, value, context):
    max_visits = 0
    max_id = 0
    
    for v in value:
        act_id = v[0]
        act_visits = int(v[1])
        if (act_visits > max_visits):
            max_visits = act_visits
            max_id = act_id
    context.write(max_id, max_visits)

job1 = Job(input_dir, output_dir1, fmap1, fred1)
job1.setCombiner(fcom1)
job1.waitForCompletion()

job2 = Job(output_dir1, output_dir2, fmap2, fred2)
job2.waitForCompletion()


True